In [1]:
import os
import multiprocessing
multiprocessing.cpu_count()

4

In [2]:
# Reference: https://mxnet.apache.org/versions/1.9.1/api/faq/env_var

# os.environ["MXNET_CPU_WORKER_NTHREADS"] = "1"
# os.environ["MXNET_CPU_PRIORITY_NTHREADS"] = "1"
# os.environ["MXNET_CPU_NNPACK_NTHREADS"] = "1"
# os.environ["MXNET_EXEC_NUM_TEMP"] = "1"
# os.environ["MXNET_ENGINE_TYPE"] = "NaiveEngine"

os.environ["MXNET_CPU_WORKER_NTHREADS"] = "4"
os.environ["MXNET_CPU_PRIORITY_NTHREADS"] = "4"
os.environ["MXNET_CPU_NNPACK_NTHREADS"] = "4"
os.environ["MXNET_EXEC_NUM_TEMP"] = "4"
os.environ["MXNET_ENGINE_TYPE"] = "ThreadedEnginePerDevice"

In [3]:
from importlib import reload
import mxnet as mx
import time
from tqdm.notebook import tqdm

ctx = mx.cpu()

## Creation of Matrices

In [4]:
wait_for_operations = True

start_time = time.time()

for _ in tqdm(range(100)):
    a = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        a.wait_to_read()
    b = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        b.wait_to_read()
    c = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        c.wait_to_read()
    d = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        d.wait_to_read()


print("Time (s):", time.time() - start_time)

  0%|          | 0/100 [00:00<?, ?it/s]

[19:54:31] /work/mxnet/src/engine/engine.cc:54: MXNet start using engine: ThreadedEnginePerDevice


Time (s): 10.879605054855347


In [5]:
wait_for_operations = False
compute_results = False

start_time = time.time()

for _ in tqdm(range(100)):
    a = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        a.wait_to_read()
    b = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        b.wait_to_read()
    c = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        c.wait_to_read()
    d = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        d.wait_to_read()

    if compute_results:
        a.wait_to_read()
        b.wait_to_read()
        c.wait_to_read()
        d.wait_to_read()
        
print("Time (s):", time.time() - start_time)

  0%|          | 0/100 [00:00<?, ?it/s]

Time (s): 0.042786598205566406


In [6]:
wait_for_operations = False
compute_results = True

start_time = time.time()

for _ in tqdm(range(100)):
    a = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        a.wait_to_read()
    b = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        b.wait_to_read()
    c = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        c.wait_to_read()
    d = mx.nd.random.normal(shape=(1000, 1000))
    if wait_for_operations:
        d.wait_to_read()

    if compute_results:
        a.wait_to_read()
        b.wait_to_read()
        c.wait_to_read()
        d.wait_to_read()
        
print("Time (s):", time.time() - start_time)

  0%|          | 0/100 [00:00<?, ?it/s]

Time (s): 19.67207169532776


## Multiplication of Matrices

In [7]:
wait_for_operations = True
compute_results = False

start_time = time.time()

for _ in tqdm(range(10)):
    a = mx.nd.random.normal(shape=(10000, 10000))
    if wait_for_operations:
        a.wait_to_read()
    b = mx.nd.random.normal(shape=(10000, 10000))
    if wait_for_operations:
        b.wait_to_read()
    c = mx.nd.random.normal(shape=(10000, 10000))
    if wait_for_operations:
        c.wait_to_read()
    d = mx.nd.random.normal(shape=(10000, 10000))
    if wait_for_operations:
        d.wait_to_read()

    s1 = a * a * a * a * b * b * b * b
    if wait_for_operations:
        s1.wait_to_read()
    s2 = c * c * c * c * d * d * d * d
    if wait_for_operations:
        s2.wait_to_read()
    s1 = s1 * s1 * s1 * s1 * s1 * s1 * s1 * s1
    if wait_for_operations:
        s1.wait_to_read()
    s2 = s2 * s2 * s2 * s2 * s2 * s2 * s2 * s2
    if wait_for_operations:
        s2.wait_to_read()
    s_total = s1 * s2
    s_total = s_total * s_total * s_total * s_total
    if wait_for_operations:
        s_total.wait_to_read()
        
    if compute_results:
        s_total.wait_to_read()
        
print("Time (s):", time.time() - start_time)

  0%|          | 0/10 [00:00<?, ?it/s]

Time (s): 135.26983547210693


In [8]:
start_time = time.time()

s_total = []

for _ in tqdm(range(10)):
    a = mx.nd.random.normal(shape=(10000, 10000))
    b = mx.nd.random.normal(shape=(10000, 10000))
    c = mx.nd.random.normal(shape=(10000, 10000))
    d = mx.nd.random.normal(shape=(10000, 10000))

    s1 = a * a * a * a * b * b * b * b
    s2 = c * c * c * c * d * d * d * d
    s1 = s1 * s1 * s1 * s1 * s1 * s1 * s1 * s1 
    s2 = s2 * s2 * s2 * s2 * s2 * s2 * s2 * s2
    s_total = s1 * s2
    s_total = s_total * s_total * s_total * s_total

mx.nd.waitall()
        
print("Time (s):", time.time() - start_time)

  0%|          | 0/10 [00:00<?, ?it/s]

Time (s): 111.36750531196594
